In [2]:
!pip install transformers
!pip install torch
!pip install torchmetrics
!pip install pytorch-lightning


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip
  Using cached torchmetrics-0.10.0-py3-none-any.whl (529 kB)

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip
  Using cached pytorch_lightning-1.7.7-py3-none-any.whl (708 kB)
  Using cached tensorboard-2.10.1-py3-none-any.whl (5.9 MB)
  Using cached pyDeprecate-0.3.2-py3-none-any.whl (10 kB)
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-manylinux2010_x86_64.whl (4.9 MB)
  Using cached protobuf-3.19.6-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 kB)
  Using cached Markdown-3.4.1-py3-none-any.whl (93 kB)
  Using cached google_auth-2.13.0-py2.py3-none-any.whl (174 kB)
 

In [1]:
from trainer import LargeLanguageModelTrainer
from lightning_modules import SimilarityModule
from data_utils import PhraseDataset
from transformer_class import BaseTransformerModel
import torch
import pytorch_lightning as pl
import transformers

/opt/conda/lib/python3.7/site-packages/pkg_resources/__init__.py:119: PkgResourcesDeprecationWarning: 4.0.0-unsupported is an invalid version and will not be supported in a future release
  PkgResourcesDeprecationWarning,


In [9]:
pretrained_model = BaseTransformerModel(model_name="bert-base-uncased")
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-uncased")
train = PhraseDataset("processed/train.csv", tokenizer)
val = PhraseDataset("processed/valid.csv", tokenizer)
optimizer = torch.optim.Adam(pretrained_model.parameters(), lr=1e-5)
loss = torch.nn.MSELoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, factor=0.5)
callbacks = [
    pl.callbacks.ModelCheckpoint(monitor="val_loss", save_top_k=1),
    pl.callbacks.EarlyStopping(monitor="val_loss", patience=5),
    pl.callbacks.LearningRateMonitor(logging_interval="step"),
    pl.callbacks.ModelSummary(max_depth=2),
    pl.callbacks.ModelPruning(
            pruning_fn="l1_unstructured",
            amount=0.01,
            use_global_unstructured=True,
        ),
    
]
model_hyperparams  = {
    "loss": loss,
    "optimizer": optimizer,
    "scheduler": None,
    "lr": 1e-5
}
trainer_hyperparams = {
    'device': "cuda",
    'batch_size': 32,
    'num_workers': 4,
}
logger = pl.loggers.TensorBoardLogger("tb_logs", name="similarity_adam_1e-5_32_LROnPlateau", log_graph=True)
model = SimilarityModule(model=pretrained_model, hyperparameters=model_hyperparams)
trainer = LargeLanguageModelTrainer(model=model,
                                    hyperparameters=trainer_hyperparams,
                                    train_dataset=train,
                                    eval_dataset=val,
                                    callbacks=callbacks,
                                    logger=logger,
                                )
trainer.train_similarity()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `Mo

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 1.14 GiB (GPU 0; 14.76 GiB total capacity; 11.29 GiB already allocated; 501.75 MiB free; 13.36 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF